In [1]:
import requests
from tqdm import tqdm
import time

# 카카오 API

- 키워드로 장소 검색: https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-keyword
- 카카오 개발자 API 키 발급: https://developers.kakao.com/
  - App 생성
  - REST API 키 복사, 붙여넣기

In [7]:
app_key = 'KakaoAK ' + 'ac220588aa994372f4482942281b92a3'

url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

params = {
    'query': '은행',
    'category_group_code': 'BK9', # 은행
    'page': 45
}

headers = {
    'Authorization': app_key
}

# json 형태로 받겠다
res = requests.get(url, params=params, headers=headers).json()

print(res.keys())

dict_keys(['documents', 'meta'])


In [8]:
res['documents']

[{'address_name': '서울 강남구 역삼동 736-6',
  'category_group_code': 'BK9',
  'category_group_name': '은행',
  'category_name': '금융,보험 > 금융서비스 > 은행 > 신한은행',
  'distance': '',
  'id': '10613842',
  'phone': '02-552-4173',
  'place_name': '신한은행 역삼역금융센터',
  'place_url': 'http://place.map.kakao.com/10613842',
  'road_address_name': '서울 강남구 테헤란로 146',
  'x': '127.035419658988',
  'y': '37.4999791613738'},
 {'address_name': '경기 수원시 팔달구 매산로1가 18',
  'category_group_code': 'BK9',
  'category_group_name': '은행',
  'category_name': '금융,보험 > 금융서비스 > 은행 > KB국민은행',
  'distance': '',
  'id': '8827144',
  'phone': '031-240-1911',
  'place_name': 'KB국민은행 수원역지점',
  'place_url': 'http://place.map.kakao.com/8827144',
  'road_address_name': '경기 수원시 팔달구 덕영대로 924',
  'x': '127.0004284084096',
  'y': '37.26620118232164'},
 {'address_name': '서울 용산구 갈월동 92',
  'category_group_code': 'BK9',
  'category_group_name': '은행',
  'category_name': '금융,보험 > 금융서비스 > 은행 > KB국민은행',
  'distance': '',
  'id': '8208081',
  'phone': '1

In [9]:
res['meta']

{'is_end': True,
 'pageable_count': 45,
 'same_name': {'keyword': '은행', 'region': [], 'selected_region': ''},
 'total_count': 42784}

# 한계
- 45페이지까지의 값밖에 받아올 수 없음

In [10]:
params = {
    'query': '은행',
    'page': 46
}

res = requests.get(url, params=params, headers=headers).json()

print(res)

{'errorType': 'InvalidArgument', 'message': 'page is more than max'}


---

# 한계 극복

# TinyDB

In [11]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')

# 분할 검색

- 우리나라 지도를 위경도 0.25 단위의 작은 사각형으로 쪼개어 검색
- 상세페이지에서 가격 긁어오기 https://place.map.kakao.com/main/v/9847579
- IP 차단 피하기 위해 5초 대기
- DB 저장
  - place_name : 지점이름
  - road_address_name: 전체 도로명 주소
  - address_name : 전체 지번 주소
  - x : X 좌표 혹은 경도(longitude)
  - y : Y 좌표 혹은 위도(latitude)
  - category_name

In [12]:
import sqlite3

DB_FILENAME = "bank.db"
conn = sqlite3.connect(DB_FILENAME)
cur = conn.cursor()

# 테이블 만들기
cur.execute("DROP TABLE IF EXISTS bank_info")
cur.execute(f"""CREATE TABLE bank_info(
                address_name VARCHAR,
                category_name VARCHAR,
                place_name VARCHAR,
                road_address_name VARCHAR,
                x VARCHAR,
                y VARCHAR
                );""")


In [13]:
start_x = 126 # 126.0 - 130.0
start_y = 33 # 33.0 - 39.0

dx = 0.25
dy = 0.25

for i in tqdm(range(18)):
    for j in range(26):
        x = start_x + dx * i
        y = start_y + dy * j
        print(x, y)

        for page in range(1, 46):
            params = {
                'query': '은행',
                'page': page,
                'rect': f'{x},{y},{x + dx},{y + dy}'
            }

            res = requests.get(url, params=params, headers=headers).json()

            if len(res['documents']) == 0:
                break

            for doc in res['documents']:
                address_name = doc['address_name']
                category_name = doc['category_name']
                place_name = doc['place_name']
                road_address_name = doc['road_address_name']
                x = doc['x']
                y = doc['y']

                cur.execute(f"""INSERT INTO bank_info(address_name, category_name, place_name, road_address_name, x, y) VALUES('{address_name}', '{category_name}', '{place_name}', '{road_address_name}', '{x}', '{y}');""")
                conn.commit()
                time.sleep(5)

            if res['meta']['is_end']:
                break

  0%|          | 0/18 [00:00<?, ?it/s]

126.0 33.0
126.0 33.25
126.0 33.5
126.0 33.75
126.0 34.0
126.0 34.25
126.0 34.5
126.0 34.75
126.0 35.0
126.0 35.25
126.0 35.5
126.0 35.75
126.0 36.0
126.0 36.25
126.0 36.5
126.0 36.75
126.0 37.0
126.0 37.25
126.0 37.5
126.0 37.75
126.0 38.0
126.0 38.25
126.0 38.5
126.0 38.75
126.0 39.0


  6%|▌         | 1/18 [03:12<54:34, 192.61s/it]

126.0 39.25
126.25 33.0


  6%|▌         | 1/18 [04:27<1:15:52, 267.77s/it]


TypeError: can only concatenate str (not "float") to str

# 저장된 결과 확인